In [3]:
import joblib

# Modeli kaydetme
model_filename = r"..\Model\random_forest_model.joblib"

# Modeli yükleme
loaded_model = joblib.load(model_filename)
print("Model başarıyla yüklendi.")


Model başarıyla yüklendi.


In [7]:
import joblib
import librosa
import numpy as np

# Scaler ve model dosyalarını yükleme
scaler_filename = '..\\Model\\scaler.joblib'  # Modelin scaler dosyasının yolu
model_filename = '..\\Model\\random_forest_model.joblib'  # Model dosyasının yolu

scaler = joblib.load(scaler_filename)
model = joblib.load(model_filename)

def extract_features_for_prediction(file_path):
    """Ses dosyasından özellikleri çıkar."""
    try:
        # Ses dosyasını yükle
        y, sr = librosa.load(file_path, sr=None)
        
        # MFCC (Mel-frequency cepstral coefficients) çıkarımı
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)

        # Chroma çıkarımı (müzikal tonlar)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_mean = np.mean(chroma, axis=1)

        # RMS Energy çıkarımı
        rms = librosa.feature.rms(y=y)
        rms_mean = np.mean(rms)

        # Zero-Crossing Rate çıkarımı
        zcr = librosa.feature.zero_crossing_rate(y=y)
        zcr_mean = np.mean(zcr)
        
        # Özellikleri birleştir
        features = np.hstack((mfcc_mean, chroma_mean, rms_mean, zcr_mean))
        return features

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def predict_audio_with_scaler(file_path):
    """Ses dosyasını scaler ve model ile test et."""
    # Özellik çıkarma
    features = extract_features_for_prediction(file_path)
    if features is None:
        print("Özellik çıkarma başarısız.")
        return

    # Özellikleri normalize et
    features = features.reshape(1, -1)  # Modelin tahmin yapabilmesi için 2D array haline getir
    normalized_features = scaler.transform(features)

    # Model ile tahmin yap
    prediction = model.predict(normalized_features)
    print(f"Tahmin edilen etiket: {prediction[0]}")

# Test edilecek ses dosyası
test_file = "C:\\VoiceProject\\Voice\\irem\\korku\\Iremkorku1.wav"  # Test ses dosyasının tam yolunu yazın
predict_audio_with_scaler(test_file)


Tahmin edilen etiket: irem


c:\Users\iremm\.conda\envs\BTK-AI\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [21]:
import librosa
import numpy as np
import joblib
import sounddevice as sd
import librosa.effects

# Model ve scaler dosyalarını yükleme
scaler_filename = '..\\Model\\scaler.joblib'  # Modelin scaler dosyasının yolu
model_filename = '..\\Model\\random_forest_model.joblib'
model = joblib.load(model_filename)

# Eğitim sırasında belirlenmiş min ve max değerler
FEATURE_MIN = np.array([0] * 27)  # Eğitim verilerinizden gelen minimum değerler
FEATURE_MAX = np.array([1] * 27)  # Eğitim verilerinizden gelen maksimum değerler

# Ses kaydetme fonksiyonu
def record_audio(duration=5, fs=16000):
    """Mikrofondan ses kaydet."""
    print("Ses kaydediliyor...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()
    print("Ses kaydı tamamlandı.")
    return recording.flatten()

# Gürültü azaltma fonksiyonu
def reduce_noise(y, sr=16000):
    """Ses kaydına gürültü azaltma uygulama."""
    try:
        # Librosa'yı kullanarak gürültü azaltma (bu, basit bir filtreleme yöntemidir)
        y_denoised = librosa.effects.preemphasis(y)  # Preemphasis genellikle frekans bandında gürültüyü azaltır
        return y_denoised
    except Exception as e:
        print(f"Gürültü azaltma sırasında hata oluştu: {e}")
        return y

# Özellik çıkarma fonksiyonu
def extract_features_for_prediction(y, sr=16000):
    """Ses verisinden özellikleri çıkar."""
    try:
        # MFCC (Mel-frequency cepstral coefficients) çıkarımı
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)

        # Chroma çıkarımı (müzikal tonlar)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_mean = np.mean(chroma, axis=1)

        # RMS Energy çıkarımı
        rms = librosa.feature.rms(y=y)
        rms_mean = np.mean(rms)

        # Zero-Crossing Rate çıkarımı
        zcr = librosa.feature.zero_crossing_rate(y=y)
        zcr_mean = np.mean(zcr)
        
        # Özellikleri birleştir
        features = np.hstack((mfcc_mean, chroma_mean, [rms_mean], [zcr_mean]))  # RMS ve ZCR tek değer
        return features

    except Exception as e:
        print(f"Hata oluştu: {e}")
        return None

# Özellikleri normalize etme
def normalize_features(features):
    """Manuel olarak özellikleri 0 ile 1 arasına çek."""
    if features.shape[0] != FEATURE_MIN.shape[0]:
        print(f"Özellik boyutları uyuşmuyor: features={features.shape[0]}, min/max={FEATURE_MIN.shape[0]}")
        return None
    try:
        normalized_features = (features - FEATURE_MIN) / (FEATURE_MAX - FEATURE_MIN)
        return normalized_features
    except Exception as e:
        print(f"Normalizasyon sırasında hata oluştu: {e}")
        return None

# Tahmin yapma fonksiyonu
def predict_audio(audio_data, sr=16000):
    """Manuel normalizasyon ile tahmin yap."""
    # Gürültü azaltma
    audio_data_denoised = reduce_noise(audio_data, sr)
    
    # Özellik çıkarma
    features = extract_features_for_prediction(audio_data_denoised, sr)
    if features is None:
        print("Özellik çıkarma başarısız.")
        return

    # Özellikleri manuel olarak normalize et
    normalized_features = normalize_features(features)
    if normalized_features is None:
        print("Normalizasyon başarısız.")
        return

    # Özellikleri yeniden şekillendir
    normalized_features = normalized_features.reshape(1, -1)  # Modelin tahmin yapabilmesi için 2D array haline getir

    # Model ile tahmin yap
    prediction = model.predict(normalized_features)
    print(f"Tahmin edilen etiket: {prediction[0]}")

# Mikrofonla ses kaydedip tahmin yapma
audio_data = record_audio(duration=3, fs=16000)  # 3 saniye ses kaydı
predict_audio(audio_data)


Ses kaydediliyor...
Ses kaydı tamamlandı.
Tahmin edilen etiket: irem
